In [8]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [9]:
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10


True

In [10]:

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("muhammadhassan.ai@muhammadhassan.tech")  # Change to your verified sender
    to_email = To("muhammad.hassan.519570@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [11]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [12]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-5.2-chat-latest"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-5.2-chat-latest"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-5.2-chat-latest"
)

In [13]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplifying SOC 2 Compliance Without the Headache

Hi {{First Name}},

I’m reaching out because many growing teams we speak with are struggling to keep SOC 2 compliance on track while still focusing on building their product.

ComplAI helps companies prepare for and maintain SOC 2 compliance faster and with far less manual work. Our AI‑powered platform automates evidence collection, maps controls to your systems, and keeps you continuously audit‑ready—so you’re not scrambling when customers or auditors ask for reports.

Teams typically use ComplAI to:
- Cut SOC 2 prep time by 50%+
- Reduce reliance on spreadsheets and manual screenshots
- Stay continuously compliant as their stack and team evolve

If SOC 2 is on your roadmap (or already a pain point), I’d be happy to show you how companies like yours are handling it more efficiently.

Would it make sense to schedule a quick 15‑minute intro next week?

Best regards,  
{{Your Name}}  
{{Title}} | ComplAI  
{{Website}}  
{{Contac

In [14]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )
outputs = [result.final_output for result in results]
for i, output in enumerate(outputs, start=1):
    print(f"\n\n--- Sales Agent {i} Output ---\n")
    print(output)




--- Sales Agent 1 Output ---

Subject: Simplify SOC 2 Compliance and Be Audit‑Ready Faster

Hi {{First Name}},

I’m reaching out because many growing teams tell us that SOC 2 compliance is becoming more time‑consuming and disruptive as they scale—especially when audits come around.

At **ComplAI**, we help companies streamline SOC 2 compliance with an AI‑powered platform that:
- Continuously monitors controls and evidence  
- Automates policy management and risk assessments  
- Keeps you audit‑ready year‑round, not just at audit time  

Teams typically reduce audit prep time by weeks while gaining clearer visibility into their compliance posture.

If SOC 2 is on your roadmap (or already a recurring headache), I’d be happy to share how ComplAI could fit into your current process. Would a quick 15‑minute conversation next week make sense?

Best regards,  
{{Your Name}}  
{{Title}} | ComplAI  
{{Email}} | {{Website}}


--- Sales Agent 2 Output ---

**Subject:** SOC 2 without the migrain

In [15]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-5.2-chat-latest"
)


In [16]:
message = "Write a cold sales email"

with trace("Pick the best cold email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )
    outputs = [result.final_output for result in results]
    emails = "\n\n".join(f"Option {i+1}:\n{output}" for i, output in enumerate(outputs))
    picker_input = f"Here are three cold sales email options:\n\n{emails}\n\nPick the best one."
    picker_result = await Runner.run(sales_picker, input=picker_input)
    print("\n\n--- Selected Best Cold Email ---\n")
    print(picker_result.final_output)



--- Selected Best Cold Email ---

**Subject:** SOC 2 without the scramble

Hi {{First Name}},

Preparing for SOC 2 doesn’t have to mean spreadsheets, consultants, and last‑minute stress.

ComplAI automates evidence collection, maps controls, and keeps you audit‑ready year‑round—with AI doing the heavy lifting. Teams typically cut prep time by 50%+ and pass audits faster.

Open to a 15‑minute chat to see if this fits {{Company}}?

Best,  
{{Your Name}}  
ComplAI  
{{Contact Info}}


In [17]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-5.2-chat-latest"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-5.2-chat-latest"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-5.2-chat-latest"
)

In [18]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-5.2-chat-latest', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [19]:
@function_tool
def send_email_tool(subject: str, body: str) -> Dict[str, str]:
    """
    Sends an email using SendGrid.
    Args:
        subject (str): The subject of the email.
        body (str): The body content of the email.
    Returns:
        Dict[str, str]: A dictionary with the status of the email sending operation.
    """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY')) 
    from_email = Email("muhammadhassan.ai@muhammadhassan.tech")  
    to_email = To("muhammad.hassan.519570@gmail.com") 
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": f"Email sent with status code {response.status_code}"}

In [20]:
send_email_tool

FunctionTool(name='send_email_tool', description='Sends an email using SendGrid.\nArgs:\n    subject (str): The subject of the email.\n    body (str): The body content of the email.\nReturns:\n    Dict[str, str]: A dictionary with the status of the email sending operation.', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'body': {'title': 'Body', 'type': 'string'}}, 'required': ['subject', 'body'], 'title': 'send_email_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72709900c9a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [21]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email_tool]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72709ba01b20>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72709ba01bc0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='

In [22]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-5.2-chat-latest")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [29]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-5.2-chat-latest")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-5.2-chat-latest")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [30]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("muhammadhassan.ai@muhammadhassan.tech")  # Change to your verified sender
    to_email = To("muhammadhassan@muhammadhassan.tech")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [31]:
tools = [subject_tool, html_tool, send_html_email]

In [32]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-5.2-chat-latest",
    handoff_description="Convert an email to HTML and send it")

In [33]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72709ba01b20>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72709ba01bc0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', description='Wr

In [34]:
## Improved Sales Manager Instructions
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)